In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = 'https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html'
browser.visit(url)

In [5]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Get the columns name and columns number in the table
#columns_block=soup.find('tr')

columns_list=soup.find_all('th')
columns=[]
for column in columns_list:
    columns.append(column.text)

col_num=len(columns)
print(columns)
print(col_num)

i=0
for i in range(col_num):
    columns[i]=[]
    print(columns[i])

""" row_blocks = soup.find_all('tr',class_='data_row')

# Make a empty list and insert the dictionary result
for row in row_blocks:
    datas =row.find_all('td')
    j=0
    for data in datas:
        print(data)
        columns[j].append(data.text)
    j=j+1 """



['id', 'terrestrial_date', 'sol', 'ls', 'month', 'min_temp', 'pressure']
7
[]
[]
[]
[]
[]
[]
[]


" row_blocks = soup.find_all('tr',class_='data_row')\n\n# Make a empty list and insert the dictionary result\nfor row in row_blocks:\n    datas =row.find_all('td')\n    j=0\n    for data in datas:\n        print(data)\n        columns[j].append(data.text)\n    j=j+1 "